# Advanced Lane Finding 


### Objective:

The main of this Advanced Lane Finding project is to,

* Identify both **yellow and white Lane** Lines in different conditions like roads with shadows and color gradient.

* Rejection of **Shadows** and, variation in lane and environment colors shall be done using HLS filter and Sobel Operator (Absolute, Magnitude and Direction filters).

* The image which was processed to identify the lanes shall be **free from distortion**. Distortion shall be avoided by using the **calibration matrix** obtained from calibrating the camera with the provided **Chess board images**.

* The Lane pixels shall be identified using the image in **Bird’s Eye view**(Top View) and not from normal view. This rectified image shall be obtained by using **Perspective transformation** on the original image with predefined Source and Destination points.

* Identify the **curvature of the Lane** (in meters) and **vehicle position** with respect to center. These details shall be displayed in the final image.

* Merge the identified Lane boundaries (with color) into the original image.

In [116]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML
get_ipython().run_line_magic('matplotlib', 'inline')
%matplotlib inline

## Camera Calibration and Disortion Correction - Using ChessBoard Images

In [117]:
"""Identify the corners of the provided chessboard images"""
def IdentifyChessBoardCorners(Nx = 9,Ny = 6):
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)s

    objp = np.zeros((Nx*Ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:Nx,0:Ny].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')
    # Step through the list and search for chessboard corners
    for fname in images:
        img = mpimg.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (Nx,Ny),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (Nx,Ny), corners, ret)

    return objpoints, imgpoints, gray


## Calibrate Camera

In [118]:
"""Calibrate the camera using the identified corner points a
and return the Camera matrix and distortion coefficients"""

def CalibrateCamera(objpoints,imgpoints,gray):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return ret, mtx, dist

## Disrotion Correction

In [119]:
"""Undistort the provided image using the distortion coefficient"""
def DistortionCorrec(img,mtx,dist):
    
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    
    return dst

## Thresholded Binary Image

## Color Transforms - Filter S and L channels

In [120]:
"""The L and S channels are used by applying channel filter and limiting values with threshold."""
def GetColorTransform(image, S_thresh, L_thresh):
    
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    
    
    S_Filter = (s_channel > S_thresh[0]) & (s_channel <= S_thresh[1])
    L_Filter = (l_channel > L_thresh[0]) & (l_channel <= L_thresh[1])
    
    return S_Filter,L_Filter



## Gradient - Execute Absolute and Directional Sobel operations

## Absolute Sobel

In [121]:
"""Get the bianry image with absolute sobel filter applied on the input image"""
def abs_sobel_thresh(gray, orient='x', thresh_min=0, thresh_max=255):
    
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    else:
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    return binary_output

## Binary Thresholded Image - Combination of All

In [124]:
def CreateBinaryThresholdedImage(image,Schannel, LChannel):

    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    
    gradx = abs_sobel_thresh(gray,'x', 30, 200)
    grady = abs_sobel_thresh(gray,'y', 30, 200)

    combined = np.zeros_like(image[:,:,0])
    combined[((gradx == 1) & (grady == 1) | ((Schannel == 1) & (LChannel ==1)))] = 255

    return combined

## Mask the Lane region

In [125]:
"""Mask the region of interest"""
def MaskLaneRegion(BinaryImage,region_of_interest_vertices):

    mask = np.zeros_like(BinaryImage)
    cv2.fillPoly(mask, [region_of_interest_vertices], 1)
    thresholded = cv2.bitwise_and(BinaryImage, mask)

    return thresholded

## Perspective Transform (Bird's Eye View)

In [126]:
"""Get the perspective transformed image of the input image"""
def PerspectiveTransfrm(thresholded, image,src,dst):

    img_size = (thresholded.shape[1], thresholded.shape[0])

    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)

    warped = cv2.warpPerspective(thresholded, M, img_size,flags=cv2.INTER_LINEAR)

    NewImage = image.copy()
    srcpts = np.array(src, np.int32)
    srcpts = srcpts.reshape((-1,1,2))
    cv2.polylines(image,[srcpts],True,(0,255,0), thickness=5)

    return warped, M_inv

## Identify Lane Pixels - Using Histogram of Warped Image

In [127]:
def find_lane_pixels(binary_warped):

    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        
        #Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin 
        win_xleft_high = leftx_current + margin  
        win_xright_low = rightx_current - margin 
        win_xright_high = rightx_current + margin  
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(255,255,0), 5) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(255,0,0), 5)  

        #Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &  (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &  (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ##If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty, out_img

## Identify a Polynomial Curve

In [128]:
def fit_polynomial(binary_warped):
    # Find our lane pixels first
    global lastleft_fit,lastright_fit
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    #Fit a second order polynomial to each using `np.polyfit` ###
    if lefty.size != 0 and leftx.size != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
        if lastleft_fit!= None and np.average(np.abs(left_fit - lastleft_fit))>5 :
            left_fit = lastleft_fit
        lastleft_fit = left_fit
    else:
        left_fit = lastleft_fit
        
    if righty.size != 0 and rightx.size != 0:
        right_fit = np.polyfit(righty, rightx, 2)
        if lastright_fit!= None and np.average(np.abs(right_fit - lastright_fit))>5 :
            right_fit = lastright_fit
        lastright_fit = right_fit
    else:
        right_fit = lastright_fit
        

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty


    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]
    
    return left_fit, right_fit

In [129]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    ##Fit a second order polynomial to each with np.polyfit() ###
    global oldleft_fit, oldright_fit
    #Fit a second order polynomial to each using `np.polyfit` ###
    if lefty.size != 0 and leftx.size != 0:
        left_fit = np.polyfit(lefty, leftx, 2)
        oldleft_fit = left_fit
    else:
        left_fit = oldleft_fit
        
    if righty.size != 0 and rightx.size != 0:
        right_fit = np.polyfit(righty, rightx, 2)
        oldright_fit = right_fit
    else:
        right_fit = oldright_fit


    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    #Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

## Search Around earlier Lines

In [130]:
def search_around_poly(binary_warped,left_fit,right_fit):

    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit new polynomials
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)

    ## Visualization ##
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

    return result, left_fitx, right_fitx

## Identify Radius of Curvature and Center of Vehicle

In [131]:
"""Calculate the radius of curvature and the position of vehicle with respect to center of lane"""
def measure_curvature_real(leftx,rightx):
    ym_per_pix = 30/720 # meters per pixel in y dimension
    #xm_per_pix = 3.7/700 # meters per pixel in x dimension
    lane_width = rightx[-1] - leftx[-1]
    xm_per_pix = 3.7 / lane_width
    # If no pixels were found return None
    ploty = np.linspace(0, 720-1, 720)
    y_eval = np.max(ploty)
    
    left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)

    # Calculation of R_curve (radius of curvature)
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad


## Plot the Lane on Actual Image

In [132]:
def DrawIdentifiedLane(image,warped,left_fit_cr,right_fit_cr,M_inv,radiusStr,DistfrmCentStr):

    out_img = np.dstack((warped, warped, warped))*255
    y_points = np.linspace(0, 719, 720)
    
    left_line_window = np.array(np.transpose(np.vstack([left_fit_cr, y_points])))
    right_line_window = np.array(np.flipud(np.transpose(np.vstack([right_fit_cr, y_points]))))
    
    line_points = np.vstack((left_line_window, right_line_window))
    cv2.fillPoly(out_img, np.int_([line_points]), [0,255, 0])
    img_size = (out_img.shape[1], out_img.shape[0])
    
    unwarped = cv2.warpPerspective(out_img, M_inv, img_size , flags=cv2.INTER_LINEAR)
    result = cv2.addWeighted(image, 1, unwarped, 0.3, 0)
    cv2.putText(result,radiusStr , (50, 50), cv2.FONT_HERSHEY_SIMPLEX , 1.0, (255,255,255), thickness=2)
    cv2.putText(result, DistfrmCentStr, (50, 100), cv2.FONT_HERSHEY_SIMPLEX , 1.0, (255,255,255), thickness=2)

    return result

## Project Pipeline 

1. Identify **Chess board corners**.
2. Calibrate Camera by identifying **Camera matrix and distortion coefficients**.
3. **Undistort** the image.
4. Choose **Color Channels** which needs to be activated for proper identification of lanes in all conditions.
5. Form a binary threshold values from using **Gradient thresholds** in the Direction of X and Directional Gradient.
6. **Mask** the Lane region.
7. Transform the Binary Image into Warped image (Bird’s Eye view) using **Perspective Transform**.
8. Fit **Polynomial Curve** by Identifying the Lane Pixels using **Sliding Window method**.
9. In order to reduce the processing of the lane pixel detection, use the **last searched** and identified pixel data.
10. Find the **curvature of lane**.
11. Calculate the **center of Vehicle** with respect to the Lane center.
12. Display the **identified lane image** over the **Actual image** along with proper text to display curvature and vehicle center.

In [ ]:
def Project_PipeLine(image):
    global OnlyOnce
    global mtx
    global dist

    # Camera Calibration
    if OnlyOnce == "yes":
        # 1. Chess Board Corner Detection
        objpoints, imgpoints, gray = IdentifyChessBoardCorners(9,6)
        # 2. Calibrate Camera
        ret, mtx, dist = CalibrateCamera(objpoints,imgpoints,gray)
        OnlyOnce = "no"

    # 3. UnDistort the Image
    
    dst = DistortionCorrec(image,mtx,dist)

    # 4. Split Different Color Channels
    S_thresh = [100,255]
    L_thresh = [125,255]
    Schannel, LChannel = GetColorTransform(dst,S_thresh,L_thresh)
    
    # 5. Form Binary Thresholded Image
    BinaryImage = CreateBinaryThresholdedImage(dst,Schannel, LChannel)
    height,width = BinaryImage.shape
    
    # 6. Mask the region
    region_of_interest_vertices = np.array([[0,height-1], [width/2, height/2], [width-1, height-1]], dtype=np.int32)
    thresholded = MaskLaneRegion(BinaryImage,region_of_interest_vertices)
    
    #7. Perspective Transform
    
    #Source Points
    SLeftBottom = [180,720]
    SLeftTop    = [540,470]
    SRightBottom = [1150,720]
    SRightTop    = [730,470]

    srcpt = np.float32([SLeftBottom,SRightBottom,SRightTop,SLeftTop])

    # Destination Points
    DLeftBottom = [350,720]
    DLeftTop    = [350,10]
    DRightBottom = [800,720]
    DRightTop    = [800,10]

    dstpt = np.float32([DLeftBottom,DRightBottom,DRightTop,DLeftTop])
    
    warped,M_inv = PerspectiveTransfrm(thresholded, dst,srcpt,dstpt)
    
    
    # 8. Fit Polynomial
    left_fit, right_fit = fit_polynomial(warped)
    
    #9. Search Around
    result, left_fit_cr, right_fit_cr = search_around_poly(warped,left_fit,right_fit)
    
    # 10. Identify Lane Curvature
    left_curverad, right_curverad = measure_curvature_real(left_fit_cr,right_fit_cr)
    avg_curverad  = (left_curverad + right_curverad) /2
    radiusStr = str("Radius of Lane Curvature = " + str(avg_curverad) + 'm')
    if mode == "Image":
        print("Radius of Left Curvature = " , left_curverad , 'm')
        print("Radius of Right Curvature = " , right_curverad , 'm')
        print("Radius of Lane Curvature = " , avg_curverad , 'm')
        
    # 11. Vehicle Center 
    lane_width = abs(left_fit_cr[-1] - right_fit_cr[-1])
    xm_per_pix = 3.7 / lane_width
    Lane_Center = ((left_fit_cr[719]) + (right_fit_cr[719])) / 2 
    
    Offset = ((dst.shape[1]/2) - Lane_Center)
    DistfrmCent = xm_per_pix*Offset
    
    if DistfrmCent < 0:
        DistfrmCentStr = str("Vehicle is " + str(abs(DistfrmCent)) +  "m right from center")
    else:
        DistfrmCentStr = str("Vehicle is " + str(abs(DistfrmCent)) +  "m left from center")
        
    if mode == "Image":
        print("Vehicle is ", abs(DistfrmCent), "m from center")
        
    # 12. Obtain final image with lane
    Result = DrawIdentifiedLane(image,warped,left_fit_cr,right_fit_cr,M_inv,radiusStr,DistfrmCentStr)

    return Result    

## Pipeline for Project Video

In [135]:
mode = "Video"
OnlyOnce = "yes"
mtx = None
dist = None
lastleft_fit = None
lastright_fit = None
oldleft_fit = None
oldright_fit = None
output = 'project_video_output.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(Project_PipeLine)
get_ipython().run_line_magic('time', 'white_clip.write_videofile(output, audio=False)')

[MoviePy] >>>> Building video output_images/project_video_output.mp4
[MoviePy] Writing video output_images/project_video_output.mp4


100%|█████████▉| 1260/1261 [05:49<00:00,  3.71it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_images/project_video_output.mp4 

CPU times: user 3min 39s, sys: 3.12 s, total: 3min 42s
Wall time: 5min 52s
